In [ ]:
!pip install qiskit==1.4.2
!pip install qiskit_machine_learning==0.8.2
!pip install qiskit_algorithms==0.3.0
!pip install openpyxl
!pip install XlsxWriter
!pip install pylatexenc
!pip install qiskit_ibm_runtime
!pip install qiskit_ibm_transpiler

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
QiskitRuntimeService.save_account() #Poner Token aqui

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import ParameterVector
from qiskit_ibm_runtime import QiskitRuntimeService, Batch, SamplerV2 as Sampler
from qiskit_algorithms.utils import algorithm_globals
from qiskit.result import marginal_counts
from qiskit_machine_learning.datasets import ad_hoc_data
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

random_seed = 12345
algorithm_globals.random_seed = random_seed
feature_dimension = 2  # Ad-Hoc tiene dimensión 2
n_qubits = feature_dimension

TRAIN_SIZE = 30
TEST_SIZE = 10

X_train_full, y_train_full, X_test_full, y_test_full = ad_hoc_data(
    training_size=TRAIN_SIZE,
    test_size=TEST_SIZE,
    n=feature_dimension,
    gap=0.3,
    one_hot=False,
    include_sample_total=False,
)

X_full = np.vstack((X_train_full, X_test_full))
y_full = np.hstack((y_train_full, y_test_full))

# Escalado
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_scaled = scaler.fit_transform(X_full)

# Etiquetas en {-1, 1}
y_full = np.where(y_full == 0, -1, 1)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_full, train_size=TRAIN_SIZE, test_size=TEST_SIZE, random_state=random_seed)

print(f"Dataset Ad-Hoc cargado. Train: {X_train.shape}, Test: {X_test.shape}")


# CONFIGURACIÓN BACKEND IBM
service = QiskitRuntimeService(name=" ") #Poner nombre
backend = service.backend("ibm_sherbrooke")  # o el que prefieras
print(f"Backend seleccionado: {backend.name}")

def build_custom_feature_map(n_qubits):
    x = ParameterVector('x', n_qubits)
    qc = QuantumCircuit(n_qubits, n_qubits)
    for i in range(n_qubits):
        qc.h(i)
        qc.rz(x[i], i)
    for i in range(n_qubits - 1):
        qc.cx(i, i + 1)
    return qc, x

feature_map, params = build_custom_feature_map(n_qubits)
# Visualizar el circuito de codificación (feature map)
print("\nFeature Map Cuántico generado:")
print(feature_map.draw(fold=120, idle_wires=False))


def build_kernel_matrix_real_ibm(X, feature_map, params, backend, shots=512, batch_size=1000):
    n_samples = X.shape[0]
    zero_state = '0' * n_qubits

    transpiled_circuits = []
    for x in X:
        bound = feature_map.assign_parameters(dict(zip(params, x)))
        transpiled = transpile(bound, backend=backend, basis_gates=backend.target.operation_names, optimization_level=1)
        transpiled_circuits.append(transpiled)

    print(transpiled_circuits[0].draw(fold=120, idle_wires=False))

    composed_circuits = []
    for i in range(n_samples):
        for j in range(n_samples):
            composed = transpiled_circuits[i].compose(transpiled_circuits[j].inverse())
            composed.measure(range(n_qubits), range(n_qubits))
            composed = transpile(composed, backend=backend, basis_gates=backend.target.operation_names, optimization_level=1)
            composed_circuits.append(composed)


    fidelities = []
    with Batch(backend=backend, max_time="3m") as batch:
        sampler = Sampler(mode=batch)
        jobs = []
        for i in range(0, len(composed_circuits), batch_size):
            job = sampler.run(composed_circuits[i:i+batch_size], shots=shots)
            jobs.append(job)

        for job in jobs:
            result = job.result()
            for pub_result in result:
                data_bin = pub_result.data
                reg_name = list(data_bin.__dict__.keys())[0]
                counts = getattr(data_bin, reg_name).get_counts()
                marginal = marginal_counts(counts, indices=list(range(n_qubits)))
                fid = marginal.get(zero_state, 0) / shots
                fidelities.append(fid)

    return np.array(fidelities).reshape(n_samples, n_samples)

def build_kernel_matrix_cross_real_ibm(X_test, X_train, feature_map, params, backend, shots=512, batch_size=1000):
    n_test = X_test.shape[0]
    n_train = X_train.shape[0]
    zero_state = '0' * n_qubits

    transpiled_test = []
    for x in X_test:
        bound = feature_map.assign_parameters(dict(zip(params, x)))
        transpiled = transpile(bound, backend=backend, basis_gates=backend.target.operation_names, optimization_level=1)
        transpiled_test.append(transpiled)

    transpiled_train = []
    for x in X_train:
        bound = feature_map.assign_parameters(dict(zip(params, x)))
        transpiled = transpile(bound, backend=backend, basis_gates=backend.target.operation_names, optimization_level=1)
        transpiled_train.append(transpiled)

    composed_circuits = []
    for test_circ in transpiled_test:
        for train_circ in transpiled_train:
            composed = test_circ.compose(train_circ.inverse())
            composed.measure(range(n_qubits), range(n_qubits))
            composed = transpile(composed, backend=backend, basis_gates=backend.target.operation_names, optimization_level=1)
            composed_circuits.append(composed)

    fidelities = []
    with Batch(backend=backend, max_time="3m") as batch:
        sampler = Sampler(mode=batch)
        jobs = []
        for i in range(0, len(composed_circuits), batch_size):
            job = sampler.run(composed_circuits[i:i+batch_size], shots=shots)
            jobs.append(job)

        for job in jobs:
            result = job.result()
            for pub_result in result:
                data_bin = pub_result.data
                reg_name = list(data_bin.__dict__.keys())[0]
                counts = getattr(data_bin, reg_name).get_counts()
                marginal = marginal_counts(counts, indices=list(range(n_qubits)))
                fid = marginal.get(zero_state, 0) / shots
                fidelities.append(fid)

    return np.array(fidelities).reshape(n_test, n_train)

print("\nConstruyendo matriz de entrenamiento...")
K_train = build_kernel_matrix_real_ibm(X_train, feature_map, params, backend)

print("\nConstruyendo matriz de test...")
K_test = build_kernel_matrix_cross_real_ibm(X_test, X_train, feature_map, params, backend)

print("\nMatrices construidas.")

# Entrenamiento
model = SVC(kernel='precomputed')
model.fit(K_train, y_train)
y_pred = model.predict(K_test)

# Evaluación
print("\n📊 Reporte final:")
print(classification_report(y_test, y_pred, labels=[-1, 1]))



In [ ]:
import pickle

resultados = {
    "K_train": K_train,
    "K_test": K_test,
    "y_train": y_train,
    "y_test": y_test,
    "y_pred": y_pred,
    "modelo_svc": model
}

with open("resultados_kernel_cuantico_ADHOC.pkl", "wb") as f:
    pickle.dump(resultados, f)

print("\nResultados guardados en 'resultados_kernel_cuantico_ADHOC.pkl'")